In [1]:
!pip install neo4j

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.5/293.5 kB 66.5 kB/s eta 0:00:0000:0100:01


In [2]:
from neo4j import GraphDatabasephDatabase

In [20]:
GP_db=GraphDatabase.driver(uri='bolt://localhost:7687',auth=('neo4j','admin'))

In [21]:
# Function used to execute Cypher queries
def execute_query(query):
    with GP_db.session() as session:
        result = session.run(query)
        return result

In [22]:
# Cypher query to create constraints
create_constraints_query = """
CREATE CONSTRAINT unique_movie_title IF NOT EXISTS
FOR (m:Movie)
REQUIRE m.name IS UNIQUE;
"""
# Execute the constraints query
execute_query(create_constraints_query)


In [23]:
# Cypher query to load and process the CSV data
load_data_query = """
LOAD CSV WITH HEADERS FROM 'http://localhost:11001/project-e98c5b11-5b76-4c43-959d-8e3639f4088f/movies.csv' AS row
MERGE (m:Movie {title: row.name})
ON CREATE SET 
    m.releaseDate = row.released, 
    m.genre = row.genre, 
    m.budget = toFloat(row.budget), 
    m.gross = toFloat(row.gross), 
    m.runtime = toFloat(row.runtime), 
    m.country = row.country, 
    m.released_year = toInteger(row.year), 
    m.rating = row.rating
// Create Lead node only if row.star is not null
FOREACH(ignoreMe IN CASE WHEN row.star IS NOT NULL THEN [1] ELSE [] END |
    MERGE (p:Lead {name: row.star}) 
    MERGE (p)-[:ACTED_IN]->(m))

// Create Director node only if row.director is not null
FOREACH(ignoreMe IN CASE WHEN row.director IS NOT NULL THEN [1] ELSE [] END |
    MERGE (n:Director {name: row.director}) 
    MERGE (m)-[:DIRECTED_BY]->(n))

// Create Writer node only if row.writer is not null
FOREACH(ignoreMe IN CASE WHEN row.writer IS NOT NULL THEN [1] ELSE [] END |
    MERGE (k:Writer {name: row.writer}) 
    MERGE (m)-[:WRITTEN_BY]->(k))

// Create Production_House node only if row.company is not null
FOREACH(ignoreMe IN CASE WHEN row.company IS NOT NULL THEN [1] ELSE [] END |
    MERGE (l:Production_House {name: row.company}) 
    MERGE (m)-[:PRODUCED_BY]->(l))

"""

# Execute the load data query
results = execute_query(load_data_query)